In [122]:
import pandas as pd
import sklearn
import numpy as np
import json
import pandas as pd
from sklearn.covariance import EllipticEnvelope
import os
import glob
import pandas as pd
from time import sleep
from tqdm import tqdm

<h1>Import Test Dataset<h1>

In [123]:
def anomaly_detection(dataframe):
    """
    Returns a dataframe of anomalies for given timestamps. 
    Anomalies have the value -1. 
    """
    df_hr = dataframe
    rhr = df_hr["heart_rate"].median()
    # heart rate data
    df_hr = df_hr.set_index('timestamp')
    df_hr.index.name = None
    df_hr.index = pd.to_datetime(df_hr.index)
    # fit the model  ------------------------------------------------------
    model = EllipticEnvelope(random_state=10,
                                support_fraction=0.7,
                                contamination=0.1)
    model.fit(df_hr)
    # predict the test set
    preds = pd.DataFrame(model.predict(df_hr))
    preds = preds.rename(lambda x: 'anomaly' if x == 0 else x, axis=1)
    data = df_hr.reset_index()
    data = data.join(preds)
    data.reset_index().to_csv("data.csv")
    data['anomaly'] = data['anomaly'].map({-1: 1, 1: 0})
    # return data.reset_index().to_json(orient="records")

    #downsample to daily
    outputdays= data.groupby(data['index'].dt.date)['heart_rate'].quantile(0.1)
    print(outputdays)
    maximum = len(outputdays)
    for n  in range(maximum):
        if (outputdays['anomaly'][n]> 0.3):
                outputdays['anomaly'][n] = 1
        else:
            outputdays['anomaly'][n] = 0


    # for n  in range(maximum):
    #     if(n>0 and n<maximum-2):
    #         if (outputdays['anomaly'][n]== 1 and outputdays['anomaly'][n+1]==1) or (outputdays['anomaly'][n]==1 and outputdays['anomaly'][n-1]==1):
    #             outputdays['anomaly'][n] = 1
    #         else:
    #             outputdays['anomaly'][n] = 0
    #     else:
    #         outputdays['anomaly'][n] = 0
    
    return outputdays




#downsample to daily
def automata(df):
    
    pass


def filter_anomalies(df):
    df = df.loc[lambda df: df['anomaly'] == -1]
    df.reset_index().to_csv("anomalies.csv")
    return df


def lambda_handler(event, context):
    """
    Main method of the lambda function. Gets called when you send a json to the api. 
    Json will be accessible in the event variable. 
    """
    dataset_path = "test_1.csv"
    return {
        'statusCode': 200,
        'body': anomaly_detection(dataset_path)
    }


if __name__ == "__main__":
    # ret = lambda_handler(None, None)
    dataset_path = "test_1.csv"
    #df = anomaly_detection(dataset_path)
    #filter_anomalies(df)

<h1>Final Prediction<h1>

In [125]:
tmp = pd.read_csv('/Users/leonurny/Desktop/TUM_Ai_Hackathon/test_data_1.csv')
tmp.rename(columns={'0':'heart_rate', '1':'timestamp','2':'y_validate'}, inplace=True)
df = tmp.copy()
outputdays = anomaly_detection(df)

index
2026-01-05    49.0
2026-01-07    63.3
2026-01-08    52.0
2026-01-09    55.0
2026-01-10    80.9
              ... 
2026-06-10    56.0
2026-06-11    57.0
2026-06-12    53.9
2026-06-13    65.6
2026-06-15    54.0
Name: heart_rate, Length: 132, dtype: float64


KeyError: 'anomaly'

In [55]:
print('accuracy: ' + str(sklearn.metrics.accuracy_score(outputdays['y_validate'], outputdays['anomaly'])))
print('f1: ' + str(sklearn.metrics.f1_score(outputdays['y_validate'], outputdays['anomaly'],)))
print('recall: ' + str(sklearn.metrics.recall_score(outputdays['y_validate'], outputdays['anomaly'],)))
print('precision: ' + str(sklearn.metrics.precision_score(outputdays['y_validate'], outputdays['anomaly'],)))

accuracy: 0.1038961038961039
f1: 0.14814814814814817
recall: 0.08695652173913043
precision: 0.5


<h1>Export<h1>

In [56]:
outputdays.to_csv('Outputvorlage.csv')

<h1>Decision Tree<h1>

In [51]:
os.chdir("//Users/leonurny/Desktop/TUM_Ai_Hackathon/Training")
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
#combine all files in the list
tmp = pd.concat([pd.read_csv(f) for f in all_filenames ])

#tmp = pd.read_csv('/Users/leonurny/Desktop/TUM_Ai_Hackathon/test_data_1.csv')
tmp.rename(columns={'1':'h', '0':'timestamp','2':'y_validate'}, inplace=True)
df = tmp.copy()
df =df.drop('Unnamed: 0', axis =1)
df =df.set_index('timestamp', drop = True)
df

,h,y_validate
timestamp,,
2024-07-27 05:25:18,50.6,NaN
2024-07-27 05:51:50,61.0,NaN
2024-07-28 05:04:20,62.6,NaN
2024-07-28 05:29:17,64.0,NaN
2024-07-28 05:57:51,60.0,NaN
...,...,...
2029-04-11 04:11:17,72.3,NaN
2029-04-11 04:29:09,73.4,NaN
2029-04-11 04:51:03,74.0,NaN


In [52]:
for i in range(1,100):
    df['h-'+str(i)] = df['h'].shift(-i)
df =df.dropna()
from sklearn.utils import shuffle
df = shuffle(df)

In [53]:
#X, Y split
Y_Train = df['y_validate']
df = df.drop('y_validate', axis =1)
X_Train = df.copy()

In [54]:
X_Train

,h,h-1,h-2,h-3,h-4,h-5,h-6,h-7,h-8,h-9,...,h-90,h-91,h-92,h-93,h-94,h-95,h-96,h-97,h-98,h-99
timestamp,,,,,,,,,,,,,,,,,,,,,
2027-07-02 02:48:02,67.705000,71.207381,71.043651,70.380952,70.036190,72.410159,72.312698,71.213333,70.777778,70.840000,...,72.023810,69.184286,69.273333,62.763571,63.995960,76.952619,90.574286,68.018095,68.247222,68.818095
2025-12-13 05:51:02,70.198485,63.591126,65.977778,73.452056,74.753333,74.074747,75.155455,75.187518,84.741263,117.020635,...,61.961111,65.391190,64.415873,61.719769,62.308333,71.702922,93.819444,85.606061,65.909921,68.448810
2024-05-03 04:54:02,60.119048,64.754524,60.928333,59.739048,58.030952,57.520000,59.463810,68.505000,68.081667,67.108333,...,60.212857,61.625000,60.640079,60.566667,61.206349,58.333333,62.962778,59.033983,58.407619,60.943723
2026-12-03 04:22:05,49.731429,53.150000,48.560000,52.624762,53.250000,48.063333,48.063492,50.313333,47.851667,48.760000,...,53.315556,52.036111,49.757143,49.263333,49.513333,49.260000,49.817857,48.920000,48.790000,51.706667
2024-12-06 02:46:00,74.039683,73.743889,73.044048,79.776667,76.869444,73.789646,74.581183,71.158333,70.917143,70.217857,...,73.881667,73.410000,72.400794,72.031746,72.716126,71.560556,73.985000,76.537540,71.953571,74.042222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026-04-15 05:45:00,70.388254,71.343730,72.825000,73.728333,80.980556,82.583117,86.531667,83.682857,85.694048,89.308333,...,72.704762,70.269048,71.445000,72.257143,71.864286,73.682222,69.747619,75.068333,89.851905,89.795000
2026-04-14 03:25:09,96.524242,93.314286,93.758333,94.665714,99.165260,94.059697,91.160714,92.745952,95.978095,92.859798,...,99.909127,95.454545,94.657677,94.518413,95.072020,92.718254,92.482063,89.408586,94.138961,92.461111
2024-06-14 04:09:00,81.016667,82.827381,84.587302,84.075000,82.400000,85.908333,87.186905,94.932143,83.103333,78.517857,...,76.565000,74.075794,72.611429,72.106061,74.203889,76.101667,76.945000,76.659286,72.771429,73.754286


In [55]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
#clf = clf.fit(X_Train, Y_Train)

from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
#clf=RandomForestClassifier(n_estimators=100,)
clf = tree.DecisionTreeClassifier()

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_Train,Y_Train)
Y_Predict = clf.predict(X_Train)

<h1>Decision Tree performance<h1>

In [56]:
#Training set performance
print('accuracy: ' + str(sklearn.metrics.accuracy_score(Y_Train, Y_Predict)))
print('f1: ' + str(sklearn.metrics.f1_score(Y_Train, Y_Predict)))
print('recall: ' + str(sklearn.metrics.recall_score(Y_Train, Y_Predict)))
print('precision: ' + str(sklearn.metrics.precision_score(Y_Train, Y_Predict)))

accuracy: 1.0
f1: 1.0
recall: 1.0
precision: 1.0


<h1>Test set prepare<h1>

In [57]:

tmp = pd.read_csv('/Users/leonurny/Desktop/TUM_Ai_Hackathon/test_data_3.csv')
tmp.rename(columns={'heartrate':'h', 'datetime':'timestamp','covid':'y_validate'}, inplace=True)
df = tmp.copy()
df =df.drop('Unnamed: 0', axis =1)
df =df.set_index('timestamp', drop = True)
df

,h,y_validate
timestamp,,
2025-09-14 04:27:05,98,0
2025-09-14 04:47:51,76,0
2025-09-14 04:51:21,78,0
2025-09-14 04:55:06,81,0
2025-09-14 04:58:42,81,0
...,...,...
2025-12-05 05:17:35,74,1
2025-12-05 05:20:47,79,1
2025-12-05 05:28:04,106,1


In [58]:
for i in range(1,100):
    df['h-'+str(i)] = df['h'].shift(-i)
df =df.dropna()
from sklearn.utils import shuffle
df = shuffle(df)

In [59]:
#Test data
Y_Test = df['y_validate']
df = df.drop('y_validate', axis =1)
X_Test = df.copy()

In [60]:
Y_Predict_test = clf.predict(X_Test)

In [61]:
#Training set performance
print('accuracy: ' + str(sklearn.metrics.accuracy_score(Y_Test, Y_Predict_test)))
print('f1: ' + str(sklearn.metrics.f1_score(Y_Test, Y_Predict_test)))
print('recall: ' + str(sklearn.metrics.recall_score(Y_Test, Y_Predict_test)))
print('precision: ' + str(sklearn.metrics.precision_score(Y_Test, Y_Predict_test)))

accuracy: 0.47118847539015607
f1: 0.6127472527472527
recall: 0.4552580013063357
precision: 0.9368279569892473


<h1>Make final Predictions<h1>


In [120]:
os.chdir("/Users/leonurny/Downloads/testdata 2")
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
for file in all_filenames:
    original = pd.read_csv(file).copy()
    test = original.copy()
    test.rename(columns={'heart_rate':'h'}, inplace=True)
    test =test.set_index('timestamp', drop = True)

    for i in range(1,100):
        test['h-'+str(i)] = test['h'].shift(-i)
    test =test.dropna()
    from sklearn.utils import shuffle
    test = shuffle(test)

    Y_Predict_final = clf.predict(test)

    Y_final_for_merge = []
    for i in range(1,100):
        Y_final_for_merge.append(0)
        
    Y_final_for_merge = np.asarray(Y_final_for_merge)
    Y_final_for_merge = np.append(Y_final_for_merge, Y_Predict_final)

    original['prediciton'] = Y_final_for_merge

    original.to_csv(file)
    